In [32]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from data_cleaning.data_cleaning import get_zimmer, get_stockwerk, get_heizung, get_kindergarten, get_schule, get_bahn, get_miete, get_nebenkosten, get_alter, get_lage, get_kaution, get_kueche, get_bad, get_mobliert, get_quadratmeter

In [62]:
file_path = '../../data/Wohnungen_1.csv'  # Gegebenen Datensatz einlesen
data = pd.read_csv(file_path, sep=';')

# Bewohnerkategorie bestimmen
bewohnerkategorie = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']
data['Bewohnerkategorie'] = data[bewohnerkategorie].apply(
    lambda row: 'Keine' if all(row == 'nein') else ', '.join(row[row == 'ja'].index), axis=1)

# Bereinigung der Daten
data = data[data['Bewohnerkategorie'] != 'Keine']

# Spalten, in denen "nein" durch 0 und "ja" durch 1 ersetzt werden sollen
columns_to_replace = ["Hausmeister", "Garage", "Aufzug", "Balkon", "Terrasse", "Kehrwoche"]

# Ersetze "nein" und "ja" in den angegebenen Spalten
data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})

data['Zimmerzahl'] = data['Zimmerzahl'].apply(get_zimmer)
data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)
data['Heizung'] = data['Heizung'].apply(get_heizung)
data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)
data['Schule'] = data['Schule'].apply(get_schule)
data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)
data['Miete'] = data['Miete'].apply(get_miete)
data['Nebenkosten'] = data['Nebenkosten'].apply(get_nebenkosten)
data['Alter'] = data['Alter'].apply(get_alter)
data['Lage'] = data['Lage'].apply(get_lage)
data['Kaution'] = data['Kaution'].apply(get_kaution)
data['Kueche'] = data['Kueche'].apply(get_kueche)
data['Bad'] = data['Bad'].apply(get_bad)
data['Moebliert'] = data['Moebliert'].apply(get_mobliert)
data['Quadratmeter'] = data['Quadratmeter'].apply(get_quadratmeter)

#print(data.head())

C:\Users\cardu\AppData\Local\Temp\ipykernel_13864\703600614.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})


In [63]:

model = BayesianNetwork([
    ('Zimmerzahl', 'Bewohnerkategorie'),
    ('Stockwerk', 'Bewohnerkategorie'),
    ('Heizung', 'Bewohnerkategorie'),
    ('Hausmeister', 'Bewohnerkategorie'),
    ('Kindergarten', 'Bewohnerkategorie'),
    ('Schule', 'Bewohnerkategorie'),
    ('S-Bahn', 'Bewohnerkategorie')
])

# Modell fitten
model.fit(data, estimator=MaximumLikelihoodEstimator)


In [75]:
# Inferenz durchführen
inference = VariableElimination(model)

example_input_4 = {
    "Zimmerzahl": 0,        # 4-Zimmer-Wohnung
    "Stockwerk": 0,           # Gas   # Kindergarten erreichbar
    "Schule": 0,            # Schule erreichbar
}

# Wahrscheinlichkeiten für jede Zielklasse berechnen
prediction = inference.map_query(variables=["Bewohnerkategorie"], evidence=example_input_4)


print("Klassifikationsergebnis:")
print(prediction)

Eliminating: Zimmerzahl: 100%|██████████| 6/6 [00:00<00:00, 1205.84it/s]

Klassifikationsergebnis:
{'Bewohnerkategorie': 'Kleinfamilie'}
